In [1]:
from pathlib import Path
import torch
import json
import tqdm

from au2v.config import ModelConfig, TrainerConfig
from au2v.dataset_manager import load_dataset_manager
from au2v.trainer import PyTorchTrainer
from au2v.model import load_model, PyTorchModel

In [2]:
def calc_accuracy(
    model: PyTorchModel,
    num_item: int,
    test_dataset: list[tuple[int, list[int], list[int]]],
    top_k: list[int],
) -> dict[str, float]:
    hit_counts = {k: 0 for k in top_k}
    for seq_index, context_items, target_indices in tqdm.tqdm(test_dataset):
        rec_list = model.output_rec_lists(
            seq_index=torch.LongTensor([seq_index]),
            item_indices=torch.LongTensor([context_items]),
            cand_item_indices=torch.arange(num_item),
            k=max(top_k),
        )
        for k in top_k:
            hit_counts[k] += len(set(target_indices) & set(rec_list[0][:k]))

    total_rec = len(test_dataset)
    results = {
        f"Accuracy@{k}": hit_count / total_rec / k
        for k, hit_count in hit_counts.items()
    }
    return results

In [3]:
run_configs = [
    {
        "name": "User2Vec (d=16)",
        "model_name": "doc2vec",
        "d_model": 16,
        "epochs": 10,
    },
    {
        "name": "User2Vec (d=32)",
        "model_name": "doc2vec",
        "d_model": 32,
        "epochs": 10,
    },
    {
        "name": "User2Vec (d=64)",
        "model_name": "doc2vec",
        "d_model": 64,
        "epochs": 10,
    },
    {
        "name": "AU2V (wo weight-tieing)",
        "model_name": "old-attentive",
        "d_model": 64,
        "epochs": 10,
    },
    {
        "name": "AU2V (d=16)",
        "model_name": "attentive",
        "d_model": 16,
        "epochs": 10,
    },
    {
        "name": "AU2V (d=32)",
        "model_name": "attentive",
        "d_model": 32,
        "epochs": 10,
    },
    {
        "name": "AU2V (d=64)",
        "model_name": "attentive",
        "d_model": 64,
        "epochs": 10,
    },
]

In [4]:
results = {}

for run_config in run_configs:
    results[run_config["name"]] = {"accuracy": []}
    model_config = ModelConfig(
        weight_decay=1e-8,
        max_embedding_norm=1,
        d_model=run_config["d_model"],
        lr=5e-5,
    )
    trainer_config = TrainerConfig(
        dataset_name="movielens",
        model_name=run_config["model_name"],
        load_dataset=False,
        save_dataset=False,
        load_model=False,
        ignore_saved_model=True,
        epochs=run_config["epochs"],
    )

    print(model_config)
    print(trainer_config)

    dataset_manager = load_dataset_manager(
        dataset_name=trainer_config.dataset_name,
        dataset_dir=trainer_config.dataset_dir,
        data_dir="../data/",
        load_dataset=trainer_config.load_dataset,
        save_dataset=trainer_config.save_dataset,
        window_size=model_config.window_size,
    )
    print(
        "train:",
        len(dataset_manager.train_dataset),
        "valid:",
        len(dataset_manager.valid_dataset),
    )
    model = load_model(
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )
    trainer = PyTorchTrainer(
        model=model,
        dataset_manager=dataset_manager,
        trainer_config=trainer_config,
        model_config=model_config,
    )

    def on_epoch_end(epoch: int):
        result = calc_accuracy(
            model=model,
            num_item=dataset_manager.num_item,
            test_dataset=dataset_manager.test_datasets["test"],
            top_k=[10, 20, 30, 40, 50],
        )
        print(epoch, result)
        results[run_config["name"]]["accuracy"].append(result)
        torch.save(model, f"cache/model/movielens-2/{run_config['name']}-{epoch}.pt")

    losses = trainer.fit(on_epoch_end=on_epoch_end)
    results[run_config["name"]]["loss"] = losses

    with open("result-2.json", "w") as f:
        json.dump(results, f)

ModelConfig(d_model=16, init_embedding_std=0.2, max_embedding_norm=1, window_size=5, negative_sample_size=5, lr=5e-05, weight_decay=1e-08)
TrainerConfig(model_name='doc2vec', dataset_name='movielens', epochs=10, batch_size=64, verbose=False, ignore_saved_model=True, load_model=False, save_model=True, load_dataset=False, save_dataset=False, model_dir='cache/model/', dataset_dir='cache/dataset/', device='cpu')
dataset_manager does not exist at: cache/dataset/movielens.pickle, create dataset
num_seq: 6040, num_item: 3706, num_item_meta: 28, num_seq_meta: 30, num_item_meta_types: 3, num_seq_meta_types: 3
to_sequential_data start
to_sequential_data end
train: 589192 valid: 110547


100%|██████████| 9207/9207 [00:30<00:00, 299.45it/s]


train 0.006540099319262365


100%|██████████| 1728/1728 [00:00<00:00, 2018.33it/s]


valid 0.0065346670529819175


...

train 0.0051313428499251265


100%|██████████| 1728/1728 [00:07<00:00, 238.69it/s]


valid 0.005162893924598728


100%|██████████| 1591/1591 [00:17<00:00, 89.00it/s]


9 {'Accuracy@10': 0.11697045883092394, 'Accuracy@20': 0.12554996857322437, 'Accuracy@30': 0.12960402262727844, 'Accuracy@40': 0.1349151477058454, 'Accuracy@50': 0.13963544940289127}
saved model to cache/model/movielens/attentive.pt


In [10]:
"""
User2Vec (d=16), 0.15412
User2Vec (d=32), 0.17609
User2Vec (d=64), 0.18804
AU2V (wo weight-tieing), 0.19713
AU2V (d=16), 0.20411
AU2V (d=32), 0.20859
AU2V (d=64), 0.21102
"""

for method, result in results.items():
    a = max(map(lambda r: r["Accuracy@30"], result["accuracy"]))
    print(f"{method}, {a:.5}")

User2Vec (d=16), 0.16983
User2Vec (d=32), 0.17639
User2Vec (d=64), 0.18965
AU2V (wo weight-tieing), 0.19933
AU2V (d=16), 0.20616
AU2V (d=32), 0.20943
AU2V (d=64), 0.21272
